# WALMART SALES MACHINE LEARNING

<img src="https://th.bing.com/th/id/OIG.RIg3jJoYg.oF1hY08eUq?pid=ImgGn" alt="Image" width="30%" height="30%">

## PARTIE 1 : ANALYSE EXPLORATOIRE + PREPROCESING DES DONNEES

### a) Importation des librairies et des données

In [71]:
import pandas as pd
import numpy as np

import plotly.express as px

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import  OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer, r2_score
from scipy.stats import uniform
import joblib

In [72]:
data = pd.read_csv('src/Walmart_Store_sales.csv')

### b) Exploration des données

In [73]:
data

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,18-02-2011,1572117.54,NaN,59.61,3.045,214.777523,6.858
1,13.0,25-03-2011,1807545.43,0.0,42.38,3.435,128.616064,7.470
2,17.0,27-07-2012,NaN,0.0,NaN,NaN,130.719581,5.936
3,11.0,NaN,1244390.03,0.0,84.57,NaN,214.556497,7.346
4,6.0,28-05-2010,1644470.66,0.0,78.89,2.759,212.412888,7.092
...,...,...,...,...,...,...,...,...
145,14.0,18-06-2010,2248645.59,0.0,72.62,2.780,182.442420,8.899
146,7.0,NaN,716388.81,NaN,20.74,2.778,NaN,NaN
147,17.0,11-06-2010,845252.21,0.0,57.14,2.841,126.111903,NaN
148,8.0,12-08-2011,856796.10,0.0,86.05,3.638,219.007525,NaN


In [74]:
print(f'Le jeu de données contient {data.shape[1]} caractéristiques et {data.shape[0]} échantillons')

Le jeu de données contient 8 caractéristiques et 150 échantillons


In [75]:
# Description générale du dataset
data.describe(include='all')

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,150.000000,132,1.360000e+02,138.000000,132.000000,136.000000,138.000000,135.000000
unique,NaN,85,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,19-10-2012,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN
mean,9.866667,NaN,1.249536e+06,0.079710,61.398106,3.320853,179.898509,7.598430
std,6.231191,NaN,6.474630e+05,0.271831,18.378901,0.478149,40.274956,1.577173
min,1.000000,NaN,2.689290e+05,0.000000,18.790000,2.514000,126.111903,5.143000
25%,4.000000,NaN,6.050757e+05,0.000000,45.587500,2.852250,131.970831,6.597500
50%,9.000000,NaN,1.261424e+06,0.000000,62.985000,3.451000,197.908893,7.470000
75%,15.750000,NaN,1.806386e+06,0.000000,76.345000,3.706250,214.934616,8.150000


In [76]:
# Nombre de valeurs manquantes pour chaque colonne
data.isna().sum()

Store            0
Date            18
Weekly_Sales    14
Holiday_Flag    12
Temperature     18
Fuel_Price      14
CPI             12
Unemployment    15
dtype: int64

In [77]:
# Afficher les types des colonnes
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         150 non-null    float64
 1   Date          132 non-null    object 
 2   Weekly_Sales  136 non-null    float64
 3   Holiday_Flag  138 non-null    float64
 4   Temperature   132 non-null    float64
 5   Fuel_Price    136 non-null    float64
 6   CPI           138 non-null    float64
 7   Unemployment  135 non-null    float64
dtypes: float64(7), object(1)
memory usage: 9.5+ KB


In [78]:
# Convertir la colonne Date au format date
data['Date'] = pd.to_datetime(data['Date'], dayfirst=True)

### c) Analyse exploratoire des données (EDA)

In [79]:
# Histogramme des ventes hebdomadaire

difference_semaine = (data['Date'].max() - data['Date'].min()).days // 7 + 1

px.histogram(data.sort_values('Date'),
        x = 'Date',
        y = 'Weekly_Sales',
        nbins = difference_semaine,
        title = 'Histogramme des ventes hebdomadaire')

Dans cet hgistogramme des ventes hebdomadaires, on peut tout de suite voir qu'il y a beaucoup de cases vides. Cela signifie que nous n'avons pas toutes les données de ventes, mais seulement des échantillons espacés dans le temps.

In [80]:
# Graphique de dispersion entre les ventes hebdomadaires et la température:

px.scatter(data, 
           x='Temperature', 
           y='Weekly_Sales', 
           title='Graphique de dispersion entre les ventes hebdomadaires et la température (en °F)',
           width = 800,
           height = 600)

Ce graphique de dispersion nous montre qu'il n'y a pas de tendance particulière ou de correlation entre les ventes et les témpératures.

In [81]:
# Boîte à moustaches des ventes hebdomadaires par jour férié:

px.box(data, 
       x = 'Holiday_Flag', 
       y = 'Weekly_Sales',
       title = 'Boîte à moustaches des ventes hebdomadaires par jour férié',
       width = 800,
       height = 600)

On remarque sur cette boite à moustaches que lorsqu'il y a un jour férié dans la semaine, la médiane des ventes augmente y compris les minimum et maximum. On observe une sorte de stabilité par rapport aux autres semaines.

In [82]:
# Matrice de corrélation entre les différentes variables:

corr_matrix = data.corr()
px.imshow(corr_matrix, 
          x=corr_matrix.columns, 
          y=corr_matrix.index,
          title = 'Matrice de corrélation entre les différentes variables',
          color_continuous_scale='viridis',
          width = 800,
          height = 600)

Cette matrice de corrélation nous montre qu'il y a quelques indicateurs qui pourraient influencer les ventes hebdomadaire. Entre autre on retrouve le CPI (niveau social), les températures, et le magasin en lui même.

### d) Preprocessing des données

In [83]:
# Retirer les lignes avec une valeur nulle dans la colonne Weekly_Sales
data = data.dropna(subset=['Weekly_Sales'])

# Retirer les lignes avec une valeur nulle dans la colonne Date
data = data.dropna(subset=['Date'])

# Créer les colonnes de dates
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month
data['Day'] = data['Date'].dt.day
data['Day_of_week'] = data['Date'].dt.dayofweek
data = data.drop('Date', axis = 1)

# Remplacer les NaN par 0 pour Holiday_Flag
data['Holiday_Flag'].fillna(0, inplace = True)

# Retirer les outliers pour Temperature, Fuel_Price, CPI et Unemployment
columns_outliers = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment']

means = data[columns_outliers].mean()
std_devs = data[columns_outliers].std()

lower_bounds = means - 3 * std_devs
upper_bounds = means + 3 * std_devs

valid_mask = ((data[columns_outliers] >= lower_bounds) & (data[columns_outliers] <= upper_bounds)).all(axis=1)

data = data[valid_mask]

In [84]:
# Séparation des variables et de la target
target = 'Weekly_Sales'
X = data.drop(target, axis = 1)
Y = data[target]

In [85]:
# Définition des variables (numerics et categorical)
numeric_features = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Year', 'Month', 'Day', 'Day_of_week']
categorical_features = ['Store', 'Holiday_Flag']

# Création du pipeline des variables numériques
numeric_transformer = StandardScaler()

# Création du pipeline des variables catégorielles
categorical_transformer = OneHotEncoder(drop='first')

# Définition du preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [86]:
# Séparation en 2 jeux de données : train et test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

# Preprocessings sur le train set
X_train = preprocessor.fit_transform(X_train)

# Preprocessings sur le test set
X_test = preprocessor.transform(X_test)

## PARTIE 2 : MODELE DE BASE (REGRESSION LINEAIRE)

### a) Entrainement du modèle de régression linéaire

In [87]:
# # Train model
# regressor = LinearRegression()
# regressor.fit(X_train, Y_train)

# # Enregistrement du modèle Ridge
# joblib.dump(regressor, 'src/regressor_model.pkl')

# Chargement du modèle Ridge
regressor = joblib.load('src/regressor_model.pkl')

Y_pred_regressor = regressor.predict(X_test)

### b) Résultats de la régression linéaire

In [88]:
# Affichage du R2 score
print('Score R2 training set :', regressor.score(X_train, Y_train))
print('Score R2 training set :', regressor.score(X_test, Y_test))
print()
mse_regressor = mean_squared_error(Y_test, Y_pred_regressor)
print('MSE :', mse_regressor)
print('RMSE :', np.sqrt(mse_regressor))

Score R2 training set : 0.983619043388679
Score R2 training set : 0.9584738281103983

MSE : 19274145324.976105
RMSE : 138831.35569811348


In [89]:
# Récupération des colonnes correspondantes aux coefficients de la regression :
column_names = []
for name, transformer, features_list in preprocessor.transformers_:
    if name == 'num': 
        features = features_list 
    else: 
        features = transformer.get_feature_names_out()
    column_names.extend(features) 
        
# Création du dataframe avec les coefficients par feature
coefs = pd.DataFrame(index = column_names, data = regressor.coef_.transpose(), columns=["coefficients"])

# Création du second dataframe en valeur absolue
feature_importance = abs(coefs).sort_values(by = 'coefficients')

# Plot coefficients
fig = px.bar(feature_importance, orientation = 'h', height = 600, width = 800)
fig.update_layout(showlegend = False, 
                  margin = {'l': 120},# to avoid cropping of column names
                 )
fig.show()

Le résultat de cette première régression linéaire semble pour le moment assez correct. On a un score R2 de 0.958 pour le jeu de test. On constate cependant un leger overfitting étant donné que le jeu de train possède un score R2 de 0.984.

Dans un second temps, on observe une RMSE d'environ 138.000, qui signifie qu'en moyenne, les prédictions seront au dessus ou en dessous de 138.000 $. Avec des moyennes supérieures à 1 million de chiffre d'affaires par semaine, c'est raisonnable.

On remarque dans ce dernier graphique que ce sont les magasins qui influent le plus sur le modèle. Il est difficile de généraliser une tendance selon des indicteurs extérieurs sans prendre en compte l'emplacement ou la surface du magasin par exemple. Le CPI quand à lui est bien présent dans le haut du classement, et confirme notre première analyse de la matrice de correlation.

Nous allons maintenant tenter de réduire l'overfitting avec l'utilisation des modèles Ridge et Lasso.

## PARTIE 3 : LUTTER CONTRE L'OVERFITTING

### a) Entrainement du modèle de régularisation

In [90]:
# # Régularization Ridge
# ridge_model = Ridge()

# ridge_params = {
#     'alpha': uniform(0.01, 0.1)
# }

# scorer = make_scorer(r2_score)

# ridge_random_search = RandomizedSearchCV(
#     ridge_model,
#     ridge_params,
#     n_iter = 5000,
#     cv = 5,
#     scoring = scorer,
#     n_jobs = -1,
#     random_state = 0
# )

# ridge_random_search.fit(X_train, Y_train) 

# # Enregistrement du modèle Ridge
# joblib.dump(ridge_random_search, 'src/ridge_model.pkl')

# Chargement du modèle Ridge
ridge_random_search = joblib.load('src/ridge_model.pkl')

Y_pred_ridge = ridge_random_search.predict(X_test)

In [91]:
# # Régularization Lasso
# lasso_model = Lasso()

# lasso_params = {
#     'alpha': uniform(10, 20)
# }

# scorer = make_scorer(r2_score)

# lasso_random_search = RandomizedSearchCV(
#     lasso_model,
#     lasso_params,
#     n_iter=5000,
#     cv=5,
#     scoring=scorer,
#     n_jobs=-1,
#     random_state = 0
# )

# lasso_random_search.fit(X_train, Y_train)

# # Enregistrement du modèle Ridge
# joblib.dump(lasso_random_search, 'src/lasso_model.pkl')

# Chargement du modèle Ridge
lasso_random_search = joblib.load('src/lasso_model.pkl')

Y_pred_lasso = lasso_random_search.predict(X_test)

c:\Anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.486e+11, tolerance: 2.905e+09



### b) Résultat de la régularisation

In [92]:
print("Meilleurs hyperparamètres pour Ridge :", ridge_random_search.best_params_)
print("Score R2 Ridge training set :", ridge_random_search.score(X_train, Y_train))
print("Score R2 Ridge test set :", ridge_random_search.score(X_test, Y_test))
mse_ridge = mean_squared_error(Y_test, Y_pred_ridge)
print('RMSE Ridge :', np.sqrt(mse_ridge))
print()
print("Meilleurs hyperparamètres pour Lasso :", lasso_random_search.best_params_)
print("Score R2 Lasso training set :", lasso_random_search.score(X_train, Y_train))
print("Score R2 Lasso test set :", lasso_random_search.score(X_test, Y_test))
mse_lasso = mean_squared_error(Y_test, Y_pred_lasso)
print('RMSE Lasso :', np.sqrt(mse_lasso))

Meilleurs hyperparamètres pour Ridge : {'alpha': 0.010007244963849218}
Score R2 Ridge training set : 0.9816493536578723
Score R2 Ridge test set : 0.9766980178553966
RMSE Ridge : 103997.5357066705

Meilleurs hyperparamètres pour Lasso : {'alpha': 17.313526662518367}
Score R2 Lasso training set : 0.9819372504142533
Score R2 Lasso test set : 0.9739144068956953
RMSE Lasso : 110034.01846527019


In [93]:
results = {
    'Model': ['Regressor', 'Ridge', 'Lasso'],
    'Score R2 train': [regressor.score(X_train, Y_train), ridge_random_search.score(X_train, Y_train), lasso_random_search.score(X_train, Y_train)],
    'Score R2 test': [regressor.score(X_test, Y_test), ridge_random_search.score(X_test, Y_test), lasso_random_search.score(X_test, Y_test)],
    'RMSE': [np.sqrt(mse_regressor), np.sqrt(mse_ridge), np.sqrt(mse_lasso)]
}

results_df = pd.DataFrame(results)
results_df

,Model,Score R2 train,Score R2 test,RMSE
0,Regressor,0.983619,0.958474,138831.355698
1,Ridge,0.981649,0.976698,103997.535707
2,Lasso,0.981937,0.973914,110034.018465


Les scores ci-dessus nous montrent une réduction de l'overfitting, certes faible mais notable. Le meilleur résultat est celui du modèle Ridge, il a le R2 le plus élevé pour le jeu de test, y compris que la meilleure RMSE.

## PARTIE 4 : AMELIORATIONS POSSIBLES

Voici une liste de mes propositions pour améliorer le modèle :

- Constaté dès le départ, il n'y a pas assez de données dans ce que nous a transmis Walmart. On a pu garder que 100 lignes pour le modèle, ce qui est insuffisant.

- Il faudrait également obtenir toutes les données de ventes par magasin (pas de semaine vide), de sorte à pouvoir utiiser des algorithmes de time series et prendre en compte la saisonnalité etc. Par exemple avec la librairie prophet, ou tensorflow keras.

- D'après mon expérience, il manque des informations crucial sur les magasins. Il serait utile d'obtenir la surface du magasin, sa configuration (super, hyper, proxi) ou autre pour pouvoir le prendre en compte dans le modèle